In [29]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time
# packacges that needs to be installed across all nodes:
from textblob import TextBlob 


# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.197:7077") \
        .appName("jayant_uc_1")\
        .config("spark.executor.cores",2)\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled", True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","60s")\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/Reddit.comments") \
        .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/Reddit.comments") \
        .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

In [30]:
path1 = "hdfs://130.238.28.151:9000/user/ubuntu/Reddits_Comments"
path2 = 'hdfs://130.238.28.151:9000/user/hadoop/RC_2010-01'
rc_df = spark_session.read\
    .option("header", "true")\
    .json(path2)\
    .cache()


rcb_df = rc_df.select('body')
rcb_df = rcb_df.dropna()
rcb_df.show(2)


+--------------------+
|                body|
+--------------------+
|Good rant, stop l...|
|  Sounds good to me.|
+--------------------+
only showing top 2 rows



In [ ]:
# Cleaning text before analysis

rcb_df = rcb_df.filter(rcb['body'] == '[deleted]')

In [32]:


def sentiment_fnc(text):
    return TextBlob(text).sentiment.polarity #gives the polarity of the sentiment, [-1.0, 1.0]
    

sentiment_udf = udf(lambda x: sentiment_fnc(x)) 
spark_session.udf.register("sentiment_udf", sentiment_udf)
rcb_df = rcb_df.withColumn('sentiment_score',sentiment_udf('body').cast('double'))


In [ ]:
rcb_df.orderBy(asc("sentiment_score")).show()

rcb_df.orderBy(desc("sentiment_score")).show()

In [28]:
spark_context.stop()

Sentiment(polarity=1.0, subjectivity=1.0)
